In [14]:
from libraries import MyDAQ
from libraries import functions as H
import matplotlib.pyplot as plt
import numpy as np
import nidaqmx as dx
from scipy.signal import sawtooth, square
import libraries
from libraries import PMConverter
from scipy.fft import rfft, rfftfreq, irfft
import time
import random
import string
all_list = [H.RC_C]

daq = MyDAQ()
daq.name = 'MyDAQ1'
daq.samplerate = 200000
read_channel = 'AI0'	
write_channel = 'AO0'

In [15]:
PM = PMConverter()
daq = MyDAQ()

In [24]:
daq.name = 'MyDAQ1'
daq.samplerate = int(200000)
read_channel = 'AI0'	
write_channel = 'AO1'
capacity = 1000
carrierFrequency = 20000
sps = int(daq.samplerate/capacity)
print('sps: ', sps)
amplitude = 2
#transfer function. UItzoeken hoe normaal werkt (telefoon), 
length = 0  # desired string length
random_string = ''.join(random.choices(string.ascii_letters, k=length))
text = "aaaaaaaaaa" #+ random_string
print(text)
binary = PM.text_to_binary(text)
print('samples: ', len(binary) * sps)
print('duration: ', len(binary) * sps / daq.samplerate)
fm_signal = PM.binary_to_pm(binary, fmType=1)
#print(fm_signal)

configuringSignal = PM.configuring_signal(daq.samplerate, sps, 32)
print(configuringSignal)

voltages = PM.pm_to_voltage_array(fm_signal, sps, f=carrierFrequency, fs=daq.samplerate, A=amplitude, pmConfigurationSignal=configuringSignal)

zeroes = np.zeros(1)
voltages = np.concatenate([voltages, zeroes])

sps:  200
aaaaaaaaaa
samples:  17800
duration:  0.089
[  0 180   0 180   0 180   0 180   0 180   0 180   0 180   0 180   0 180
   0 180   0 180   0 180   0 180   0 180   0 180   0 180   0 180   0 180
   0 180   0 180   0 180   0 180   0 180   0 180   0 180   0 180   0 180
   0 180   0 180   0 180   0 180   0 180]


In [26]:
print('start in 3 sec')
time.sleep(3)
print('gooooo')
daq.write(voltages, write_channel)

start in 3 sec
gooooo


In [11]:
daq.name = 'MyDAQ1'
daq.samplerate = int(1000)
read_channel = 'AI0'	
write_channel = 'AO0'
carrierFrequency = 20000
sps = 100
amplitude = 2
#transfer function. UItzoeken hoe normaal werkt (telefoon), 
text = "aaaaaaaaaaAppelflappenkkkkkkkkkkkkkkkjhgkhgcvblkjgftuyseryiuygfcgfcgvhm,nxztretrertlukyeuyytkuygvcjghyukhgvcty6775i86oilki7r56tiyukljgdfguklhftrywuyturtgfjdfghuiy78i67yu"
binary = PM.text_to_binary(text)
#print(binary)
fm_signal = PM.binary_to_pm(binary, fmType=1)
#print(fm_signal)
configuringSignal = PM.configuring_signal(daq.samplerate, sps, 10)
#print(configuringSignal)
voltages = PM.pm_to_voltage_array(fm_signal, sps, f=carrierFrequency, fs=daq.samplerate, A=amplitude, pmConfigurationSignal=configuringSignal)


In [8]:
daq.write(voltages, write_channel)

In [ ]:
PM.configure_signal(voltages, daq.samplerate, carrierFrequency, sps, 10)


_, _, phases = PM.retrieve_phases(voltages, daq.samplerate, sps, carrierFrequency)
print('phases: ', np.round(np.array(phases) / np.pi * 180))

binary_back = PM.pm_to_binary(np.rad2deg(phases), fmType=1)
print("Recovered Binary:", binary_back)

text_back = PM.binary_to_text(binary_back)
print("Recovered Text:", text_back)

In [ ]:
daq.name = 'MyDAQ1'
daq.samplerate = int(2e5)
read_channel = 'AI0'	
write_channel = 'AO0'
carrierFrequency = 2e4
sps = 20000
amplitude = 2

text = "Hi!"
binary = PM.text_to_binary(text)
fm_signal = PM.binary_to_pm(binary, fmType=1)

voltages = PM.pm_to_voltage_array(fm_signal, sps, f=carrierFrequency, fs=daq.samplerate, A=amplitude)


In [ ]:
print(len(voltages))
#data = daq.readwrite(voltages, read_channel, write_channel)
data = voltages
window = np.hanning(len(data))
windowed_data = data * window
fft = np.fft.rfft(windowed_data)
freqs = np.fft.rfftfreq(len(data), 1 / daq.samplerate)
amplitude = np.abs(fft)

# Plot
#plt.figure(figsize=(10, 6))
#plt.scatter(freqs, amplitude, s=10, color='darkblue', alpha=0.7, label='FFT Amplitude')
#plt.axvline(carrierFrequency)
#lt.xscale('log')
#plt.yscale('log')  # Optional: helps when amplitudes span multiple orders
#plt.xlabel('Frequency (Hz)')
#plt.ylabel('Amplitude')
#plt.title('Frequency Spectrum (FFT)')
#plt.grid(True, which='both', linestyle='--', linewidth=0.5)
##plt.legend()
#plt.tight_layout()
#plt.show()

_, amplitudes, phases = PM.retrieve_phases(data, daq.samplerate, sps, carrierFrequency)

print(len(phases), len(fm_signal))

binary_back = PM.pm_to_binary(phases, fmType=1)
print("Recovered Binary:", binary_back)

text_back = PM.binary_to_text(binary_back)
print("Recovered Text:", text_back)

In [ ]:
plt.scatter(np.linspace(0, len(phases), len(phases)), phases)
plt.show()

In [ ]:
tArray = np.linspace(0, len(voltages) / daq.samplerate, len(voltages), endpoint=False)
print(voltages.shape)
data = daq.readwrite(voltages, read_channel, write_channel)
tArrayd = np.linspace(0, len(data) / daq.samplerate, len(data), endpoint=False)

print(len(tArrayd), len(data), len(voltages))

plt.scatter(tArray, voltages, s=1)  # s=1 to avoid huge dots
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title('Time-Domain Signal')
plt.grid(True)
plt.tight_layout()
plt.show()
plt.scatter(tArrayd, data)
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title('Time-Domain Signal')
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
duration = 5
amplitude = 2
freq = 20000
phase = 0
_, signal = daq.generateWaveform('sine', daq.samplerate, freq, amplitude, phase, duration)
signal = []
for i in range(0, 6):
    for j in range(0, 20000):
        signal.append((i - 3))
data = daq.readwrite(np.array(signal), read_channel, write_channel)
plt.scatter(np.arange(len(signal)), signal, label='sig')
plt.scatter(np.arange(len(data)), data, label='data')
plt.legend()
plt.show()

print(len(signal) - len(data))
countt = 0
for i in range(0, 6):
    count = np.sum(np.abs(data - (i-3)) <= 0.1)
    countt += count
    print(f"{count} data points are within 0.1 V of 6 V.")
print(countt)
print(len(data))